## 데이터 로드

In [1]:
import pandas as pd
import numpy as np
import json

with open('./data/train_original.json', encoding='utf-8') as train_f:
    train_data = json.loads(train_f.read())
    train_df = pd.DataFrame(train_data['documents'])

with open('./data/valid_original.json', encoding='utf-8') as valid_f:
    valid_data = json.loads(valid_f.read())
    valid_df = pd.DataFrame(valid_data['documents'])

In [ ]:
def extract_body(article) -> str:
    art_sentence = []

    for contents in article:
        if len(contents) >= 2:
            for sub_con in contents:
                art_sentence.append(sub_con['sentence'])
                continue
        elif len(contents) == 0:
            pass
        else:
            art_sentence.append(contents[0]['sentence'])

    return art_sentence

def sentence_validation(art_sentence):
    del_sentence = []
    for sentence in art_sentence:
        if '@' in sentence or '/사진' in sentence:
            del_sentence.append(sentence)
        elif sentence[-1] != '.':
            del_sentence.append(sentence)
    for del_sen in del_sentence: 
        art_sentence.remove(del_sen)
    return art_sentence

In [146]:
extract_body(train_df['text'][21531])

['동구 16일 8개월 만에 재개장 주 3회 20여 가지 음식 판매',
 '인천시 동구는 행정안전부 지정 제9호 야시장인 ‘달빛거리 송현야시장’을 오는 16일 재개장한다.',
 '13일 구에 따르면 송현시장 상인회에서 운영하는 야시장은 동인천역 4번 출구에서 도보로 불과 2분 거리에 위치해 교통편의성이 매우 뛰어나 많은 관광객들이 찾았으며, 겨울철 휴점 이후 재정비를 거쳐 8개월 만에 손님맞이 준비를 마쳤다.',
 '올해부터는 송현시장 상인들과 사전 협의를 통해 금·토요일에 이어 일요일까지 주 3회 운영되며, 오후 6시부터 11시까지 송현시장 중앙 통로 8개 매대에서 스테이크(큐브·빠네크림), 떡닭, 해산물 떡볶이 등 20여 가지 음식이 판매될 예정이다.',
 '야시장 개장일에는 찾아가는 문화공연 행사를 개최해 즐길거리와 먹을거리가 함께하는 야간 관광명소의 시작을 알린다.',
 '또 송현야시장이 행정안전부의 야시장 활성화사업에 선정됨에 따라 관광객들을 위한 문화공연 행사를 지속적으로 개최하고, 방문객을 위한 편의 공간 조성, 스카이 미디어아트 콘텐츠 제작 등 관광객 모집에도 박차를 기할 계획이다.',
 '구 관계자는 "달빛거리 송현야시장이 지역의 전통상권 부활의 시작을 알리고 인천을 대표하는 관광명소로 거듭날 수 있도록 야시장 판매자 모집 확대 등 송현시장 상인회와 혼연일체가 돼 추진하겠다"고 말했다.',
 '최유탁 기자 cyt@kihoilbo.co.kr',
 '기호일보, KIHOILBO']

In [145]:
sentence_list = extract_body(train_df['text'][21531])
sentence_validation(sentence_list)

['인천시 동구는 행정안전부 지정 제9호 야시장인 ‘달빛거리 송현야시장’을 오는 16일 재개장한다.',
 '13일 구에 따르면 송현시장 상인회에서 운영하는 야시장은 동인천역 4번 출구에서 도보로 불과 2분 거리에 위치해 교통편의성이 매우 뛰어나 많은 관광객들이 찾았으며, 겨울철 휴점 이후 재정비를 거쳐 8개월 만에 손님맞이 준비를 마쳤다.',
 '올해부터는 송현시장 상인들과 사전 협의를 통해 금·토요일에 이어 일요일까지 주 3회 운영되며, 오후 6시부터 11시까지 송현시장 중앙 통로 8개 매대에서 스테이크(큐브·빠네크림), 떡닭, 해산물 떡볶이 등 20여 가지 음식이 판매될 예정이다.',
 '야시장 개장일에는 찾아가는 문화공연 행사를 개최해 즐길거리와 먹을거리가 함께하는 야간 관광명소의 시작을 알린다.',
 '또 송현야시장이 행정안전부의 야시장 활성화사업에 선정됨에 따라 관광객들을 위한 문화공연 행사를 지속적으로 개최하고, 방문객을 위한 편의 공간 조성, 스카이 미디어아트 콘텐츠 제작 등 관광객 모집에도 박차를 기할 계획이다.',
 '구 관계자는 "달빛거리 송현야시장이 지역의 전통상권 부활의 시작을 알리고 인천을 대표하는 관광명소로 거듭날 수 있도록 야시장 판매자 모집 확대 등 송현시장 상인회와 혼연일체가 돼 추진하겠다"고 말했다.']

In [ ]:
max_input_length = 2000
max_target_length = 750
prefix = ""

def preprocess_function(feature_list, target_list):
    inputs = [prefix + doc for doc in feature_list]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
# 데이터가 손실될 경우에 앞의 단어가 아니라 뒤의 단어가 삭제되도록 하고싶다면 truncating이라는 인자를 사용합니다. truncating='post'를 사용할 경우 뒤의 단어가 삭제됩니다.
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(target_list, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs